# First Test for EVT

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
from itertools import product

# Import functions
import backtesting as BT
import plots as IP
import evt as EVT
import data_download as dd

### Data

In [ ]:
'''# Download data
sp500_data = yf.download("^GSPC", start="2000-01-01", end="2024-01-01")
sp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))
returns = sp500_data["Log Returns"].dropna()'''

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [14]:
# Define tickers and number of shares
tickers = ["MSFT", "NVDA", "AAPL"]
shares = pd.Series({"MSFT": 3, "NVDA": 3, "AAPL": 3})

# Download price data
prices = dd.get_raw_prices(tickers, start="2020-01-01")

# Compute portfolio value each day
portfolio_value = (prices * shares).sum(axis=1)

# Compute simple daily returns
returns = portfolio_value.pct_change().dropna()

# Return as DataFrame (same format as SP500 example)
portfolio_returns_df = returns.to_frame(name="Returns")

returns = portfolio_returns_df["Returns"].dropna()

wealth = wealth = portfolio_value.iloc[-1]

print(f"Portfolio value: {wealth:.2f} USD")

print(f'\n Portfolio retuns: \n {returns.head()}')

Portfolio value: 2261.73 USD

 Portfolio retuns: 
 Date
2020-01-03   -0.011688
2020-01-06    0.004316
2020-01-07   -0.007182
2020-01-08    0.015611
2020-01-09    0.015219
Name: Returns, dtype: float64


### EVT

In [15]:
# Set parameters
confidence_level = 0.99 # <----- Can choose 0.95 etc

In [19]:
# Apply EVT model
evt_output, var_estimate, es_estimate, prob_exceedance = EVT.evt(
    returns=returns,
    confidence_level=confidence_level,
    threshold_percentile=97,
    exceedance_level=0.05,  # optional: probability of exceeding 5% loss
    wealth=wealth
)

# Print VaR and ES Estimates
print(f"EVT VaR estimate $ (abs): {var_estimate:.2f}")
print(f"EVT Expected Shortfall estimate $ (abs): {es_estimate:.2f}")

# If you also want the probability of exceeding 5% loss:
if prob_exceedance is not None:
    print(f"Probability of exceeding 5% daily loss: {prob_exceedance * 100:.2f}%")

# Plot interactive VaR
fig_var = IP.plot_backtest(evt_output, subset=("2020-01-01", "2025-01-01"), interactive=True)


EVT VaR estimate $ (abs): 111.23
EVT Expected Shortfall estimate $ (abs): 151.95
Probability of exceeding 5% daily loss: 0.94%


In [20]:
evt_output.head()

,Returns,VaR,ES,VaR Violation,VaR_monetary,ES_monetary
Date,,,,,,
2020-01-03,-0.011688,0.049178,0.067185,False,111.226944,151.954071
2020-01-06,0.004316,0.049178,0.067185,False,111.226944,151.954071
2020-01-07,-0.007182,0.049178,0.067185,False,111.226944,151.954071
2020-01-08,0.015611,0.049178,0.067185,False,111.226944,151.954071
2020-01-09,0.015219,0.049178,0.067185,False,111.226944,151.954071


In [21]:
evt_output, var_estimate, es_estimate, prob_exceedance, diag = EVT.evt(
    returns=returns,
    confidence_level=confidence_level,
    threshold_percentile=99,
    exceedance_level=0.05,
    diagnostics=True
)

print(f"xi: {diag['xi']:.4f}, beta: {diag['beta']:.4f}")
print(f"Threshold u: {diag['threshold_u']:.4f}")
print(f"Max support (x_max): {diag['max_support']:.4f}")
print(f"Number of exceedances: {diag['num_exceedances']}")


xi: 0.4100, beta: 0.0114
Threshold u: 0.0479
Max support (x_max): inf
Number of exceedances: 14


### Backtesting (Extensive)

In [22]:
total_violations, violation_rate = BT.count_violations(evt_output)

print(f"Total VaR Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")


Total VaR Violations: 14
Violation Rate: 1.04%


In [23]:
kupiec_results = BT.kupiec_test(
    total_violations=total_violations,
    total_days=len(evt_output),
    confidence_level=confidence_level
)

print("\nKupiec Test Results:")
for key, value in kupiec_results.items():
    print(f"{key}: {value}")


Kupiec Test Results:
LR_uc: 0.022417470024549857
p_value: 0.880981861600229
reject_null: False


In [24]:
christoffersen_results = BT.christoffersen_test(evt_output)

print("\nChristoffersen Test Results:")
for key, value in christoffersen_results.items():
    print(f"{key}: {value}")



Christoffersen Test Results:
LR_c: 2.250362324653537
p_value: 0.13358312168527875
reject_null: False


In [25]:
joint_results = BT.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)

print("\nJoint Test Results:")
for key, value in joint_results.items():
    print(f"{key}: {value}")


Joint Test Results:
LR_total: 2.2727797946780868
p_value: 0.3209756879034268
reject_null: False
